In [1]:
import polars as pl
import pandas as pd
from IPython.display import display

In [2]:
from Predictors.Pb.features import validate_features_vs_target

In [3]:
from data.Constants.ColumnNames import WsfrlColumnOverall
from data.Constants.ColumnNames import WsfrlColumnPriorStats
from data.Constants.ColumnNames import WsfrlColumnForAges


In [4]:
import Predictors.DataPrep as dp
df_ages = dp.get_data()

display(df_ages)

Position,Time,time_in_seconds,Bib Number,Name,Gender,Gender Position,Club,is_club_member,Points,RaceName,year,month,day,date,distance,pace,club_position,club_position_gender,is_pb,first_time_run,prior_run_attempts,prior_quickest_run_in_year,prior_quickest_run_time,prior_quickest_run_time_in_seconds,prior_quickest_run_position,prior_time_vs_current_change,prior_time_vs_current_time,prior_quickest_vs_run_time,fellow_club_mates_running,fellow_club_mates_running_male,fellow_club_mates_running_female,best_position_year_to_date,average_position_year_to_date,worst_position_year_to_date,best_points_year_to_date,average_points_year_to_date,worst_points_year_to_date,runs_in_wsfrl_year_to_date,wsfrl_tenure_in_runs,wsfrl_tenure_in_years,position_record_years_since_best,position_record_years_since_worst,position_record_best_year,position_record_worst_year,position_record_best_position_all_time,position_record_average_position_all_time,position_record_worst_position_all_time,position_record_past_pb_all_time,past_pb_year_to_date,prior_course_pbs,Gender_First_Char,p_of_10_firstname,p_of_10_surname,p_of_10_track_age_category,p_of_10_road_age_category,p_of_10_xc_age_category,p_of_10_gender,p_of_10_club,p_of_10_athlete_id,match_type,matches,parameters_used,min_age,max_age,average_age,min_dob,max_dob,avg_dob,p_of_10_track_age_category_as_of_2024,p_of_10_road_age_category_as_of_2024,p_of_10_xc_age_category_as_of_2024,IsAgeCategoryMissing,is_big_club
i64,str,f64,i64,str,str,f64,str,bool,f64,str,i64,i64,i64,str,f64,str,f64,f64,bool,bool,f64,str,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,f64,f64,f64,str,str,str,str,str,str,str,f64,str,f64,str,str,str,str,str,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64,str,str,str,bool,bool
1,"""00:27:42""",1662.0,325,"""Tom Owens""","""M""",1.0,"""Shettleston Harriers""",true,null,"""Hangover-5""",2011,1,1,"""2011-01-01""",8.0,"""0:03:27.750000""",1.0,1.0,false,true,0.0,null,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,null,0.0,"""M""",null,null,"""TBC""","""TBC""","""TBC""",null,null,null,null,null,null,null,null,null,null,null,null,"""Age""","""Age""","""Age""",true,false
218,"""00:48:20""",2900.0,328,"""Balalie Weialt""","""M""",171.0,"""Arena 80 AC""",true,4.0,"""Hangover-5""",2011,1,1,"""2011-01-01""",8.0,"""0:06:02.500000""",3.0,3.0,false,true,0.0,null,null,null,null,null,null,null,5.0,3.0,2.0,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,null,0.0,"""M""",null,null,"""TBC""","""TBC""","""TBC""",null,null,null,null,null,null,null,null,null,null,null,null,"""Age""","""Age""","""Age""",true,true
217,"""00:48:17""",2897.0,182,"""Paul Crowe""","""M""",170.0,"""Henfield Joggers""",true,4.0,"""Hangover-5""",2011,1,1,"""2011-01-01""",8.0,"""0:06:02.125000""",5.0,4.0,false,true,0.0,null,null,null,null,null,null,null,6.0,5.0,1.0,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,null,0.0,"""M""","""Paul""","""Crowe""","""V50""","""V50""","""V50""","""M""","""Henfield""",141209.0,"""match""",1.0,"""all""",52.0,56.0,54.0,1959.0,1955.0,1957.0,"""V65""","""V65""","""V65""",false,true
216,"""00:48:12""",2892.0,234,"""Nigel Mcnestrie""","""M""",169.0,"""Chichester Runners""",true,4.0,"""Hangover-5""",2011,1,1,"""2011-01-01""",8.0,"""0:06:01.500000""",22.0,16.0,false,true,0.0,null,null,null,null,null,null,null,25.0,17.0,8.0,null,null,null,null,null,null,0.0,0.0,0.0,null,null,null,null,null,null,null,0.0,null,0.0,"""M""","""Nigel""","""McNestrie""","""V45""","""V45""","""V45""","""M""","""Chichester""",322764.0,"""match""",1.0,"""all""",47.0,51.0,49.0,1964.0,1960.0,1962.0,"""V60""","""V60""","""V60""",false,true
215,"""00:48:08""",2888.0,27,"""Linda Cooper""","""F""",47.0,"""Arunners""",true,6.0,"""Hangover-5""",2011,1,1,"""2011-01-01""",8.0,"""0:06:01""",15.0,3.0,false,true,0.0,null,null,null,null,null,null,null,30.0,19.0,11.0,null,null,null,null,null,null,0.0,0.

In [5]:
# Remove all rows where the year is 2014
df_ages = df_ages.filter(
     (pl.col(WsfrlColumnForAges.AgeCategoryRoad) != "TBC")
    # & (pl.col('year') != 2014)
    & pl.col(WsfrlColumnForAges.AgeAvgRange).is_not_null()
    & pl.col(WsfrlColumnForAges.AgeMinRange).is_not_null()
    & pl.col(WsfrlColumnForAges.AgeMaxRange).is_not_null()
)


In [6]:
 # Pivot the DataFrame
pivot_df_year_by_age_cat = df_ages.pivot(
    values=[WsfrlColumnForAges.AgeCategoryRoad],
    index="year",
    on=WsfrlColumnForAges.AgeCategoryRoad,
    aggregate_function="len"
).sort("year")

display(pivot_df_year_by_age_cat)


year,V50,V45,V35,V40,V55,V65,V60,U11,SEN,U17,V70,V75,V85,U20,U13,U23,U15,V80
i64,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32,u32
2011,466,410,255,405,298,102,213,10,261,92,35,null,null,null,null,null,null,null
2012,379,405,226,263,473,207,291,9,205,null,97,36,3,null,null,null,null,null
2013,499,528,329,390,467,198,314,7,237,null,102,33,8,128,null,null,null,null
2014,720,700,565,619,605,202,471,20,450,null,146,35,13,214,7,null,null,null
2015,786,797,624,655,669,214,402,19,577,null,113,36,1,null,3,null,null,null
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2019,851,781,342,570,743,273,503,10,571,60,178,32,null,null,null,null,3,21
2020,62,58,34,44,43,14,26,2,34,null,9,2,null,5,1,null,null,null
2022,583,401,215,285,606,364,636,6,241,31,172,96,6,21,7,null,2,12


In [7]:
 # Pivot the DataFrame
# Assuming df_ages is your Polars DataFrame
pivot_df_age_cat = df_ages.pivot(
    values=[WsfrlColumnForAges.AgeCategoryRoad],  # Use "*" to count all occurrences
    index=WsfrlColumnForAges.AgeCategoryRoad,
    on=WsfrlColumnOverall.Gender,
    aggregate_function="len"
).to_pandas()

print(type(df_ages))
# # Sum the counts across gender columns to create a row total
# pivot_df = pivot_df.with_column(
#     pl.sum_horizontal(pivot_df.select(pl.col(WsfrlColumnOverall.Gender).unique())).alias("row_total")
# )
pivot_df_age_cat["All"] = pivot_df_age_cat["M"] + pivot_df_age_cat["F"] 

pivot_df_age_cat = pivot_df_age_cat.sort_values(by="p_of_10_road_age_category", ascending=False, inplace=True)



<class 'polars.dataframe.frame.DataFrame'>


In [8]:
from data.Constants.ColumnNames import WsfrlColumnForAgesGenerated

age_category_mapping = {
    "SEN": "Senior",
    "TBC": "Junior",
    "U11": "Junior",
    "U13": "Junior",
    "U15": "Junior",
    "U17": "Junior",
    "U20": "Junior",
    "U23": "Junior",
    "V35": "Senior",
    "V40": "Veteran V40",
    "V45": "Veteran V45",
    "V50": "Veteran V50",
    "V55": "Veteran V55",
    "V60": "Veteran V60",
    "V65": "Veteran V65",
    "V70": "Veteran V70+",
    "V75": "Veteran V70+",
    "V80": "Veteran V70+",
    "V85": "Veteran V70+",
    "V95": "Veteran V70+"
}

age_category_mapping_plus = {
    "SEN": "Senior",
    "TBC": "Junior",
    "U11": "Junior",
    "U13": "Junior",
    "U15": "Junior",
    "U17": "Junior",
    "U20": "Junior",
    "U23": "Junior",
    "V35": "Senior",
    "V40": "Veteran V40+",
    "V45": "Veteran V45+",
    "V50": "Veteran V50+",
    "V55": "Veteran V55+",
    "V60": "Veteran V60+",
    "V65": "Veteran V65+",
    "V70": "Veteran V65+",
    "V75": "Veteran V65+",
    "V80": "Veteran V65+",
    "V85": "Veteran V65+",
    "V95": "Veteran V65+"
}


df_ages = df_ages.with_columns((pl.col("p_of_10_road_age_category").map_elements(lambda x: age_category_mapping.get(x), return_dtype=pl.String).alias(str(WsfrlColumnForAgesGenerated.BlendedAgeCategory))))

df_ages = df_ages.with_columns((pl.col("p_of_10_road_age_category").map_elements(lambda x: age_category_mapping_plus.get(x), return_dtype=pl.String).alias(str(WsfrlColumnForAgesGenerated.BlendedAgeCategoryPlus))))



In [10]:
# Pivot the DataFrame
# Assuming df_ages is your Polars DataFrame
pivot_df_blended_age_cat = df_ages.pivot(
    values=[WsfrlColumnForAgesGenerated.BlendedAgeCategory],  # Use "*" to count all occurrences
    index=[WsfrlColumnForAgesGenerated.BlendedAgeCategoryPlus,WsfrlColumnForAgesGenerated.BlendedAgeCategory],
    on=WsfrlColumnOverall.Gender,
    aggregate_function="len"
).to_pandas()

pivot_df_blended_age_cat = pivot_df_blended_age_cat.sort_values(
    by=[WsfrlColumnForAgesGenerated.BlendedAgeCategoryPlus, WsfrlColumnForAgesGenerated.BlendedAgeCategory]
)

print(type(df_ages))
# # Sum the counts across gender columns to create a row total
# pivot_df = pivot_df.with_column(
#     pl.sum_horizontal(pivot_df.select(pl.col(WsfrlColumnOverall.Gender).unique())).alias("row_total")
# )
pivot_df_blended_age_cat["All"] = pivot_df_blended_age_cat["M"] + pivot_df_blended_age_cat["F"] 

pivot_df_blended_age_cat

<class 'polars.dataframe.frame.DataFrame'>


,blended_age_category_plus,blended_age_category,M,F,All
7,Junior,Junior,834,621,1455
2,Senior,Senior,5267,3589,8856
3,Veteran V40+,Veteran V40,3129,2455,5584
1,Veteran V45+,Veteran V45,4004,3103,7107
0,Veteran V50+,Veteran V50,4486,3352,7838
4,Veteran V55+,Veteran V55,4375,2759,7134
6,Veteran V60+,Veteran V60,3753,1728,5481
5,Veteran V65+,Veteran V65,2310,686,2996
8,Veteran V65+,Veteran V70+,1859,455,2314


## Age Category

Dilemma is that the age category is  unbalanced
- Junior are quite time
- Seniors are large and cannot be broken down further


| Blended Cat     | Sum of All | % of total |
|-----------------|------------:|-----------:|
| Junior          |       1455 |          3% |
| Senior          |       8856 |         18% |
| Veteran V40+    |       5584 |         11% |
| Veteran V45+    |       7107 |         15% |
| Veteran V50+    |       7838 |         16% |
| Veteran V55+    |       7134 |         15% |
| Veteran V60+    |       5481 |         11% |
| Veteran V65+    |       5310 |         11% |
| **Grand Total** | **48765**  |     **100%** |	


In [ ]:
# Remove all junior as they are a tiny sub population 
# I end up keeping them 
# df_ages = df_ages.filter(
#      (pl.col(WsfrlColumnForAgesGenerated.BlendedAgeCategory) != "Junior")
# )

In [11]:
target_columns = [
    str(WsfrlColumnForAges.AgeAvgRange), 
    str(WsfrlColumnForAges.AgeMinRange), 
    str(WsfrlColumnForAges.AgeMaxRange),
    str(WsfrlColumnForAges.AgeCategoryRoad),
    str(WsfrlColumnForAgesGenerated.BlendedAgeCategory),
    str(WsfrlColumnForAgesGenerated.BlendedAgeCategoryPlus)
    ]



WsfrlColumnCalculated.PriorVsRun = "UP" "Down"


In [12]:
df_ages = df_ages.to_dummies(
    [WsfrlColumnOverall.Gender, WsfrlColumnOverall.Club], drop_first=True 
)
display(df_ages)

df_ages.to_pandas()

Position,Time,time_in_seconds,Bib Number,Name,Gender_F,Gender Position,Club_Almost Athletes,Club_Arena 80 AC,Club_Arunners,Club_Ashford & District RRC,Club_Bexhill Runners,Club_BigClubs.BRIGHTON_AND_HOVE,Club_Bitton Road Runners,Club_Blackburn Harriers,Club_Blackburn Harriers & AC,Club_Bognor Tri Club,Club_Bridport Runners,Club_Brighto & Hove AC,Club_Brighton & Hove AC,Club_Brighton & Hove Canicross Runners,Club_Brighton Multisports Club,Club_Bure Valley Harriers,Club_Burgess Hill Runners,Club_C L C Striders,Club_Cambridge Univ Hare & Hounds,Club_Chichester Runners,Club_Chichester Westgate Triathlon Club,Club_City Of Hull AC,Club_City of Sailsbury,Club_Clowne Road Runners,Club_Crawley AC,Club_Crawley Run Crew,Club_Crawley Wheelers,Club_Crowborough,Club_Dartmouth Athletics Club,Club_Datchet Dashers,…,wsfrl_tenure_in_runs,wsfrl_tenure_in_years,position_record_years_since_best,position_record_years_since_worst,position_record_best_year,position_record_worst_year,position_record_best_position_all_time,position_record_average_position_all_time,position_record_worst_position_all_time,position_record_past_pb_all_time,past_pb_year_to_date,prior_course_pbs,Gender_First_Char,p_of_10_firstname,p_of_10_surname,p_of_10_track_age_category,p_of_10_road_age_category,p_of_10_xc_age_category,p_of_10_gender,p_of_10_club,p_of_10_athlete_id,match_type,matches,parameters_used,min_age,max_age,average_age,min_dob,max_dob,avg_dob,p_of_10_track_age_category_as_of_2024,p_of_10_road_age_category_as_of_2024,p_of_10_xc_age_category_as_of_2024,IsAgeCategoryMissing,is_big_club,blended_age_category,blended_age_category_plus
i64,str,f64,i64,str,u8,f64,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,u8,…,f64,f64,str,str,str,str,str,str,str,f64,str,f64,str,str,str,str,str,str,str,str,f64,str,f64,str,f64,f64,f64,f64,f64,f64,str,str,str,bool,bool,str,str
217,"""00:48:17""",2897.0,182,"""Paul Crowe""",0,170.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0.0,0.0,null,null,null,null,null,null,null,0.0,null,0.0,"""M""","""Paul""","""Crowe""","""V50""","""V50""","""V50""","""M""","""Henfield""",141209.0,"""match""",1.0,"""all""",52.0,56.0,54.0,1959.0,1955.0,1957.0,"""V65""","""V65""","""V65""",false,true,"""Veteran V50""","""Veteran V50+"""
216,"""00:48:12""",2892.0,234,"""Nigel Mcnestrie""",0,169.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,…,0.0,0.0,null,null,null,null,null,null,null,0.0,null,0.0,"""M""","""Nigel""","""McNestrie""","""V45""","""V45""","""V45""","""M""","""Chichester""",322764.0,"""match""",1.0,"""all""",47.0,51.0,49.0,1964.0,1960.0,1962.0,"""V60""","""V60""","""V60""",false,true,"""Veteran V45""","""Veteran V45+"""
215,"""00:48:08""",2888.0,27,"""Linda Cooper""",1,47.0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0.0,0.0,null,null,null,null,null,null,null,0.0,null,0.0,"""F""","""Linda""","""Cooper""","""V35""","""V35""","""V35""","""W""","""Arunners""",120588.0,"""match""",1.0,"""all""",37.0,41.0,39.0,1974.0,1970.0,1972.0,"""V50""","""V50""","""V50""",false,true,"""Senior""","""Senior"""
214,"""00:48:06""",2886.0,241,"""Gary Tompsett""",0,168.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,…,0.0,0.0,null,null,null,null,null,null,null,0.0,null,0.0,"""M""","""Gary""","""Tompsett""","""V50""","""V50""","""V50""","""M""","""Chichester""",342170.0,"""match""",1.0,"""all""",52.0,56.0,54.0,1959.0,1955.0,1957.0,"""V65""","""V65""","""V65""",false,true,"""Veteran V50""","""Veteran V50+"""
213,"""00:48:05""",2885.0,292,"""Michelle Hughes""",1,46.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,…,0.0,0.0,null,null,null,null,null,null,null,0.0,null,0.0,"""F""","""Michelle""","""Hughes""","""V40""","""V40""","""V40""","""W""","""Worthing""",312116.0,"""match""",1.0,"""all""",42.0,46.0,44.0,1969.0,1965.0,1967.0,"""V55""","""V55""","""V55""",false,true,"""Veteran V40""","""Veteran V40+"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,

,Position,Time,time_in_seconds,Bib Number,Name,Gender_F,Gender Position,Club_Almost Athletes,Club_Arena 80 AC,Club_Arunners,...,min_dob,max_dob,avg_dob,p_of_10_track_age_category_as_of_2024,p_of_10_road_age_category_as_of_2024,p_of_10_xc_age_category_as_of_2024,IsAgeCategoryMissing,is_big_club,blended_age_category,blended_age_category_plus
0,217,00:48:17,2897.0,182,Paul Crowe,0,170.0,0,0,0,...,1959.0,1955.0,1957.0,V65,V65,V65,False,True,Veteran V50,Veteran V50+
1,216,00:48:12,2892.0,234,Nigel Mcnestrie,0,169.0,0,0,0,...,1964.0,1960.0,1962.0,V60,V60,V60,False,True,Veteran V45,Veteran V45+
2,215,00:48:08,2888.0,27,Linda Cooper,1,47.0,0,0,1,...,1974.0,1970.0,1972.0,V50,V50,V50,False,True,Senior,Senior
3,214,00:48:06,2886.0,241,Gary Tompsett,0,168.0,0,0,0,...,1959.0,1955.0,1957.0,V65,V65,V65,False,True,Veteran V50,Veteran V50+
4,213,00:48:05,2885.0,292,Michelle Hughes,1,46.0,0,0,0,...,1969.0,1965.0,1967.0,V55,V55,V55,False,True,Veteran V40,Veteran V40+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48760,106,00:36:22,2182.0,320,Darren Winter,0,90.0,0,0,0,...,1974.0,1970.0,1972.0,V50,V50,V50,False,True,Veteran V50,Veteran V50+
48761,105,00:36:17,2177.0,228,Steve Jones,0,89.0,0,0,0,...,1969.0,1965.0,1967.0,V55,V55,V55,False,True,Veteran V55,Veteran V55+
48762,104,00:36:14,2174.0,160,Mark Kennard,0,88.0,0,0,0,...,1964.0,1960.0,1962.0,V60,V60,V60,False,True,Veteran V60,Veteran V60+
48763,121,00:37:29,2249.0,269,Evert Baker,0,99.0,0,0,0,...,1964.0,1960.0,1962.0,V60,V60,V60,False,True,Veteran V60,Veteran V60+


## Feature Selection




In [19]:
from Predictors.Pb.features import get_boolean_features

feature_columns_club = get_boolean_features(df_ages, 'Club_')


In [20]:

feature_columns_to_default_to_zero = [
    str(WsfrlColumnPriorStats.Tenure),
    str(WsfrlColumnPriorStats.TotalPriorRunCount),
    str(WsfrlColumnPriorStats.PositionYearsSinceBest),
    str(WsfrlColumnPriorStats.PositionYearSinceWorst),
]


feature_columns_mandatory = [
    str(WsfrlColumnOverall.Position),
    f"{WsfrlColumnOverall.Gender}_F",
    str(WsfrlColumnOverall.TimeInSeconds),
    str(WsfrlColumnOverall.IsClubMember),
    str(WsfrlColumnOverall.IsBigClub),
]



feature_columns = (feature_columns_to_default_to_zero 
                   + feature_columns_mandatory 
                   # + feature_columns_club                 
                   )




AttributeError: type object 'WsfrlColumnOverall' has no attribute 'IsClubMember'

In [ ]:

df_features = df_ages[feature_columns]
df_target = df_ages[target_columns]


validate_features_vs_target(df_features, df_target)

# Display the reduced DataFrame
# display(df_features.head())
# display(df_target.head())



## Split the Data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

SEED = 123

(
    df_feature_train,
    df_feature_test,
    df_target_train,
    df_target_test,
) = train_test_split(
    df_features,
    df_target,
    test_size=0.30,
    random_state=SEED,
)

total_rows = len(df_features)
train_rows = len(df_feature_train)
test_rows = len(df_feature_test)

test_percentage_of_total = train_rows / total_rows

print(f"Total Rows: {total_rows} Training Rows: {train_rows} Test Rows: {test_rows} Test Percentage of Total: {test_percentage_of_total:.2f}")

# train_test_split return numpy arrays, convert them back to Pandas DataFrames
# Because that is what the pipeline expects
df_feature_train.to_pandas(),
df_feature_test.to_pandas(),
df_target_train.to_pandas(),
df_target_test.to_pandas(),

validate_features_vs_target(df_feature_train,df_target_train)
validate_features_vs_target(df_feature_test,df_target_test)


In [ ]:
from Predictors.Pb.features import general_feature_validation

general_feature_validation(df_feature_train, feature_columns)

In [ ]:
from enum import Enum
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier

class RegressionMethod(Enum):
    LOGISTIC = "LogisticRegression"
    LINEAR = "LinearRegression"
    DECISIONTREE = "DecisionTree"



def build_pipeline(target_column: str, method: RegressionMethod ) -> Pipeline:
    # Create imputers for different features/columns
    default_to_zero_imputer = SimpleImputer(strategy="constant", fill_value=0) #-np.inf is too big for a float
    binary_value_imputer = SimpleImputer(strategy="constant", fill_value=np.uint8(0))
    mandatory_average_imputer = SimpleImputer(strategy="constant", fill_value=0)
    
    
    # Combine the imputers using ColumnTransformer
    preprocessor = ColumnTransformer(
        transformers=[
            ("default_zero_features", default_to_zero_imputer, feature_columns_to_default_to_zero),
            # ("club_features", binary_value_imputer, feature_columns_club),
            ("mandatory_features", mandatory_average_imputer, feature_columns_mandatory),
        ],
        # remainder="drop",
    )
    
    PIPELINE_STEP_PREPROCESSOR = "preprocessor"
    PIPELINE_STEP_SCALER = "scaler"
    PIPELINE_STEP_CLASSIFIER = "classifier"
    
    # Create a pipeline with the preprocessor, scaler, and the appropriate regression model
    steps = [
        (PIPELINE_STEP_PREPROCESSOR, preprocessor),
        (PIPELINE_STEP_SCALER, StandardScaler()),
    ]
    
    if method == RegressionMethod.LOGISTIC:
        steps.append((PIPELINE_STEP_CLASSIFIER, LogisticRegression(penalty=None, max_iter=1000)))
    elif method == RegressionMethod.LINEAR:
        steps.append((PIPELINE_STEP_CLASSIFIER, LinearRegression()))
    elif method == RegressionMethod.DECISIONTREE:
        steps.append((PIPELINE_STEP_CLASSIFIER, LinearRegression()))
    
    pipeline = Pipeline(steps=steps)

    # Fit the pipeline on the training data
    pipeline.fit(df_feature_train, df_target_train[target_column])
    return pipeline



In [ ]:
pipeline_logistic_regression = (
    build_pipeline(WsfrlColumnForAges.AgeCategoryRoad, RegressionMethod.LOGISTIC))

pipeline_logistic_regression_blended_cat = (
    build_pipeline(WsfrlColumnForAgesGenerated.BlendedAgeCategory, RegressionMethod.LOGISTIC))

pipeline_logistic_regression_blended_cat_plus = (
    build_pipeline(WsfrlColumnForAgesGenerated.BlendedAgeCategoryPlus, RegressionMethod.LOGISTIC))

pipeline_linear_regression = build_pipeline(
    WsfrlColumnForAges.AgeAvgRange, RegressionMethod.LINEAR)

In [ ]:

def run_pipeline(pipeline: Pipeline, df_feature: pd.DataFrame):
    # Transform the test data using the preprocessor part of the pipeline
    preprocessed_df = pipeline.named_steps['preprocessor'].transform(df_feature)

    # Scale the preprocessed test data
    scaled_df = pipeline.named_steps['scaler'].transform(preprocessed_df)
    
    return scaled_df
    
def run_logistic_regression_pipeline_in_stages(pipeline: Pipeline,  
                                               df_feature: pd.DataFrame, 
                                               df_target: pd.DataFrame, 
                                               target_column: str) -> tuple:
    
    # Scale the preprocessed test data
    scaled_df = run_pipeline(pipeline, df_feature)

    classifier = LogisticRegression(penalty=None, max_iter=1000)

    classifier.fit(
        scaled_df,
        df_target[target_column],
    )

    return scaled_df, classifier



## Age Category
Use logistic regression to predict the **age category** which is **very UNbalanced**

In [ ]:

(scaled_train_df_age_cat,
logistic_regression_classifier) = run_logistic_regression_pipeline_in_stages(
    pipeline_logistic_regression,
    df_feature_train,
    df_target_train,
    WsfrlColumnForAges.AgeCategoryRoad )





## blended category
Use logistic regression to predict the **blended category** which is **very balanced**

In [ ]:
(scaled_train_df_age_cat_blended,
logistic_regression_classifier_for_blended) = run_logistic_regression_pipeline_in_stages(
    pipeline_logistic_regression_blended_cat,
    df_feature_train,
    df_target_train,
    WsfrlColumnForAgesGenerated.BlendedAgeCategory )


In [ ]:
(scaled_train_df_age_cat_blended_plus,
logistic_regression_classifier_for_blended_plus) = run_logistic_regression_pipeline_in_stages(
    pipeline_logistic_regression_blended_cat_plus,
    df_feature_train,
    df_target_train,
    WsfrlColumnForAgesGenerated.BlendedAgeCategoryPlus)

In [ ]:
def run_linear_regression_pipeline_in_stages(pipeline: Pipeline,  
                                               df_feature: pd.DataFrame, 
                                               df_target: pd.DataFrame, 
                                               target_column: str) -> tuple:
    
    # Scale the preprocessed test data
    scaled_df = run_pipeline(pipeline, df_feature)

   # Create a LinearRegression model
    regressor = LinearRegression()
    
    # Fit the regressor on the scaled training data
    regressor.fit(
        scaled_df,
        df_target[target_column],
    )

    return scaled_df, regressor

In [ ]:
(scaled_train_df_avg_age,
linear_regression) = run_linear_regression_pipeline_in_stages(
    pipeline_linear_regression,
    df_feature_train,
    df_target_train,
    WsfrlColumnForAges.AgeAvgRange )

In [ ]:
import altair as alt

def display_feature_coefficients(classifier, feature_columns):
    # Convert the set to a list to maintain order
    all_feature_columns_list = list(feature_columns)
    
    
    # Create a DataFrame with the features and their coefficients
    feature_coefficients_df = pl.DataFrame(
        {
            "feature": all_feature_columns_list,
            "coefficient": classifier.coef_[0][:len(feature_columns)],
        }
    )
    
    # Sort the DataFrame by coefficient in descending order
    sorted_feature_coefficients_df = feature_coefficients_df.sort("coefficient", descending=True)
    
    # Plot the bar chart
    alt.Chart(sorted_feature_coefficients_df.to_pandas()).mark_bar().encode(
        x="coefficient",
        y=alt.Y("feature", sort="-x")
    ).properties(
        width=500,
        height=300,
    ).display()

In [ ]:
display_feature_coefficients(logistic_regression_classifier, feature_columns)

In [ ]:
display_feature_coefficients(logistic_regression_classifier_for_blended, feature_columns)

In [ ]:
display_feature_coefficients(logistic_regression_classifier_for_blended_plus, feature_columns)

In [ ]:
# Make predictions on the test data
predictions = pipeline_linear_regression.named_steps['classifier'].predict(scaled_train_df_avg_age)

## Unbalnced (age Cat)


In [ ]:
(scaled_test_df_age_cat,
junk__as_on_test_logistic_regression_classifier) = run_logistic_regression_pipeline_in_stages(
    pipeline_logistic_regression,
    df_feature_test,
    df_target_test,
    WsfrlColumnForAges.AgeCategoryRoad )

In [ ]:
(scaled_test_df_age_cat_blended,
still_junk_as_on_test_logistic_regression_classifier) = run_logistic_regression_pipeline_in_stages(
    pipeline_logistic_regression_blended_cat,
    df_feature_test,
    df_target_test,
    WsfrlColumnForAgesGenerated.BlendedAgeCategory)

In [ ]:
(scaled_test_df_age_cat_blended_plus,
still_junk_as_on_test_logistic_regression_classifier_plus) = run_logistic_regression_pipeline_in_stages(
    pipeline_logistic_regression_blended_cat_plus,
    df_feature_test,
    df_target_test,
    WsfrlColumnForAgesGenerated.BlendedAgeCategoryPlus)

In [ ]:
## Balnced (blended)

In [ ]:
from sklearn.metrics import classification_report

# Must be scaled like training data
validate_features_vs_target(scaled_test_df_age_cat, df_feature_test)

# Cat Age (unbalanced)
print(
    classification_report(
        df_target_test.get_column(WsfrlColumnForAges.AgeCategoryRoad),
        logistic_regression_classifier.predict(scaled_test_df_age_cat),
        zero_division=0
    )
)

# Blended Cat Age (balanced)
print(
    classification_report(
        df_target_test.get_column(WsfrlColumnForAgesGenerated.BlendedAgeCategory),
        logistic_regression_classifier_for_blended.predict(scaled_test_df_age_cat_blended),
        zero_division=0
    )
)

# Blended Cat Age PLUS ( high level and balanced)
print(
    classification_report(
        df_target_test.get_column(WsfrlColumnForAgesGenerated.BlendedAgeCategoryPlus),
        logistic_regression_classifier_for_blended_plus.predict(scaled_test_df_age_cat_blended_plus),
        zero_division=0
    )
)

## Really poor to predict

did not do very well

> Overall Accuracy:
    The overall accuracy is 0.20 (20%), meaning the model's predictions are correct 20% of the time.
    This is low, especially if compared to a baseline model (e.g., always predicting the most common class). 

> Classes like SEN, V45, V50, V55, and V60 dominate the dataset, while smaller classes like U11, U13, and U15 have minimal representation. Logistic regression may overly focus on larger classes, neglecting smaller ones.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree

#scaled_test_df_age_cat_blended_plus

def run_decision_tree_pipeline_in_stages(pipeline: Pipeline,  
                                               df_feature: pd.DataFrame, 
                                               df_target: pd.DataFrame, 
                                               target_column: str) -> tuple:
    
    # Scale the preprocessed test data
    scaled_df = run_pipeline(pipeline, df_feature)

    classifier = DecisionTreeClassifier(max_depth=3, random_state=SEED)
     
    classifier.fit(
        scaled_df,
        df_target[target_column],
    )
    
    return scaled_df, classifier
    


In [ ]:
(tree_scaled_train_df_age_cat_blended_plus,
 decision_tree_classifier_for_blended_plus
 ) = run_decision_tree_pipeline_in_stages(
    pipeline_logistic_regression_blended_cat_plus,
    df_feature_train,
    df_target_train,
    WsfrlColumnForAgesGenerated.BlendedAgeCategoryPlus)

In [ ]:
class_names = sorted(df_ages[WsfrlColumnForAgesGenerated.BlendedAgeCategoryPlus].unique())

plt.figure(figsize=(10, 10), dpi=300)
plot_tree(
    decision_tree_classifier_for_blended_plus, feature_names=df_feature_train.columns, class_names=class_names, filled=True
)
plt.show()

In [ ]:
(scaled_test_df_avg_age,
junk_as_on_test_linear_regression) = run_linear_regression_pipeline_in_stages(
    pipeline_linear_regression,
    df_feature_test,
    df_target_test,
    WsfrlColumnForAges.AgeAvgRange )

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Make predictions on the test data
predictions = linear_regression.predict(scaled_test_df_avg_age)

# Calculate evaluation metrics
mae = mean_absolute_error(df_target_test[WsfrlColumnForAges.AgeAvgRange], predictions)
mse = mean_squared_error(df_target_test[WsfrlColumnForAges.AgeAvgRange], predictions)
r2 = r2_score(df_target_test[WsfrlColumnForAges.AgeAvgRange], predictions)

print(f"Mean Absolute Error (MAE): {mae}")
print(f"Mean Squared Error (MSE): {mse}")
print(f"R-squared (R²): {r2}")